# TFIDF

### Libraries

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### Read data

set path of data in `PATH` variable

In [2]:
PATH = './data/IR_dataset/'
PATH = PATH.rstrip('/')

store txt files into a list named `doc`

In [13]:
docs = []
for index in range(0, 3258):
    with open(f"{PATH}/{index}.txt", 'r', encoding='utf8') as file_reader:
        docs.append(file_reader.read())

show example of data

In [4]:
for doc in docs[:3]:
    print (doc[:500])
    print('-' * 50)

برخی از هواداران مصدق یا اعضای جبهه ملی که در زمان نخست‌وزیری مصدق، از جبهه ملی یا از هیئت‌وزیران کنار گذاشته شده یا کنار رفتند، پس از جدایی از مصدق، به انتقاد از کارنامه وی پرداختند و حتی برای سرنگونی‌اش تلاش کردند. برخی از این افراد عبارت‌اند از:
* فضل‌الله زاهدی (نخست‌وزیر کودتا)
* علی امینی
* حسین مکی، (که در آغاز سرباز فداکار وطن نامیده شد ولی در پایان به دلیل مخالفت با مصدق از سوی هواداران جبهه ملی سرباز خطاکار وطن خطاب می‌شد)
* مظفر بقایی (به دلیل اتهام مشارکت در قتل سرتیپ افشار طوس و سپس
--------------------------------------------------
جبهه ملی ایران که به اختصار جبهه ملی نیز خوانده می‌شود سازمان سیاسی ملی‌گرای سکولار، دموکرات و جمهوری‌خواه فعال در ایران است، که در حال حاضر تحت رهبری دکتر موسویان فعالیت می‌کند.

جبهه ملی ایران در سال ۱۳۲۸ توسط سیاستمدارانی از قبیل محمد مصدق، حسین فاطمی و کریم سنجابی تأسیس شد و به پیش نهاد حسین فاطمی، ملی شدن صنعت نفت ایران را مطرح کرد. دکتر مصدق در کتاب خاطرات خود می‌گوید: «پیشنهاد ملی‌شدن صنعت نفت در سراسر کشور ابتکار شادروان دکتر حسین فاطمی

In [5]:
## data clean

### TFIDF TOP K Relevent Documents 

In [6]:
## function: tokenizer persian 

In [7]:
## list: stop word persian

controllable parameters: `min_df`, `max_df`, `max_feautures`, `ngram_range`

In [8]:
# Initialize a vectorizer
model_tfidf = TfidfVectorizer(analyzer="word", min_df = 1, max_df=0.5, max_features=10000, ngram_range=(1,2))
# Create a corpus of documents and convert to TFIDF vectors
matrix = model_tfidf.fit_transform(docs)

In [9]:
# https://mccormickml.com/2021/05/27/question-answering-system-tf-idf/
def tfidf_top_k(query, model_tfidf, matrix, k=2):

    query_tfidf = model_tfidf.transform([query])
    
    # Stores cosine similarity scores
    doc_scores = []

    # Compute the cosine similarity scores
    for doc in matrix:
        doc_scores.append(cosine_similarity(query_tfidf, doc)[0][0])

    # Sort list of doc_scores and return the top k indices of highest scores
    sorted_scores = sorted(enumerate(doc_scores), key=lambda ind_score: ind_score[1], reverse=True)
    top_doc_indices = [ind for ind, score in sorted_scores[:k]]
  
    return top_doc_indices

In [10]:
query = "آدولف هیتلر شکست و مرگ"

In [11]:
tfidf_top_k(query, model_tfidf, matrix, k=20)

[484,
 482,
 344,
 345,
 343,
 349,
 481,
 341,
 466,
 1544,
 342,
 347,
 463,
 354,
 220,
 443,
 469,
 483,
 442,
 449]